In [1]:
import pandas as pd
from sklearn.decomposition import NMF
from utils.process import prepare_df, data_arrays
from sklearn import metrics
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = prepare_df()
df = df[df['Label'] != 'BENIGN']
df = df.sample(10000)
df.groupby('Label')['Label'].count()

Label
Bot                           40
DDoS                        2286
DoS GoldenEye                180
DoS Hulk                    4153
DoS Slowhttptest              94
DoS slowloris                117
FTP-Patator                  121
PortScan                    2836
SSH-Patator                  135
Web Attack   Brute Force      24
Web Attack   XSS              14
Name: Label, dtype: int64

In [3]:
class SSNM_wrapper:

    def fit_transform(self, X, **kwargs):
        self.model = SSNMF(X, **kwargs)
        self.model.mult(1000)
        return self

    def fit(self, X, **kwargs):
        return self.fit_transform(X, **kwargs)

    def score(self, X, y):
        y_pred = self.predict()
        return metrics.rand_score(y, y_pred)

    def predict(self):
        return np.argmax(self.model.S, axis=0)



In [4]:
from ssnmf import SSNMF
import random


def get_Y(X, y):
    y_unique = list(np.unique(y))
    sample_size = X.shape[1]
    Y = np.zeros((len(y_unique), sample_size))
    for i in range(sample_size):
        j = y_unique.index(y[i])
        Y[j, i] = 1
    return Y


def get_L(Y, fraction_known=0.5):
    L = np.zeros(Y.shape)
    m, n = L.shape
    labeled_data = random.sample(range(n), int(fraction_known*n))
    for i in labeled_data:
        L[:i] = 1
    return L



#model = SSNMF(M_s,10, modelNum=1)

In [5]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from ssnmf import SSNMF

skf = StratifiedKFold(n_splits=5)

X, y = data_arrays(df)
skf.get_n_splits(X, y)


k = 12
lam = 50
fraction_known = 0.1

for lam in (0, 0.001, 0.5, 0.1, 0.5, 1, 10, 100, 500, 1000, 5000):
    for train_index, test_index in skf.split(X, y):
        X_train_T, X_test_T = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        Y_train = get_Y(X_train_T.T, y_train)
        L_train = get_L(Y_train, fraction_known=fraction_known)
        #print(f"norm X {np.linalg.norm(X_train_T)}")
        snmf = SSNMF(X_train_T.T, k, L=L_train, Y=Y_train, lam=lam * np.linalg.norm(X_train_T), modelNum=5)
        snmf.mult(numiters=1000)
        y_pred = np.argmax(snmf.S, axis=0)
        score = metrics.rand_score(y_train, y_pred)
        print(f'lambda {lam} score {score}')


# #

/home/tosia/NASK/network_analyze/utils/process.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace([np.inf, -np.inf], np.nan, inplace=True)


lambda 0 score 0.8014728182662744
lambda 0 score 0.7878843784940482
lambda 0 score 0.8026308782288277
lambda 0 score 0.7956489319183337
lambda 0 score 0.7655660331427642
lambda 0.001 score 0.770901405107937
lambda 0.001 score 0.785680014701414
lambda 0.001 score 0.77610269951855
lambda 0.001 score 0.8056294111978765
lambda 0.001 score 0.7819999616788557
lambda 0.5 score 0.7907881842380747
lambda 0.5 score 0.8252469325109523
lambda 0.5 score 0.8208043670413225
lambda 0.5 score 0.8676479066649981
lambda 0.5 score 0.8538705455203367
lambda 0.1 score 0.8206980463350289
lambda 0.1 score 0.8841195729665278
lambda 0.1 score 0.8292222084224536
lambda 0.1 score 0.8301532686500861
lambda 0.1 score 0.8280967175426974
lambda 0.5 score 0.8512570805458839
lambda 0.5 score 0.8338871865043332
lambda 0.5 score 0.8437654295018784
lambda 0.5 score 0.8360157370983012
lambda 0.5 score 0.8164893371415756
lambda 1 score 0.859765627565038
lambda 1 score 0.828999513042368
lambda 1 score 0.8150182144901487
lamb

In [ ]:
print(metrics.completeness_score(Labels, np.argmax(model.S, axis=0)))
print(metrics.homogeneity_score(Labels, np.argmax(model.S, axis=0)))

array([[83868909.0, 3675920.0, 986.0, ..., 95502143.0, 166023.0,
        102585716.0],
       [8.0, 4.0, 7.0, ..., 8.0, 4.0, 13.0],
       [7.0, 0.0, 0.0, ..., 4.0, 0.0, 6.0],
       ...,
       [50000000.0, 0.0, 0.0, ..., 59600000.0, 0.0, 0.0],
       [76400000.0, 0.0, 0.0, ..., 89800000.0, 0.0, 99200000.0],
       [5738681.0, 0.0, 0.0, ..., 5557884.0, 0.0, 99200000.0]],
      dtype=object)